In [2]:
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt

In [3]:
metadata_file = "C:/Users/spigno/Documents/GitHub/PopulationData/MetagenomicData/MetadataMetgendata.csv"  # Percorso file metadati
metagenomic_data_file = "C:/Users/spigno/Documents/GitHub/PopulationData/MetagenomicData/RelabMetgendata.csv"  # Percorso file dati metagenomici
# Caricare i dataset
metadata = pd.read_csv(metadata_file)
metagenomic_data = pd.read_csv(metagenomic_data_file)

In [22]:

# Filtrare i metadati per i criteri specificati
filtered_metadata = metadata[
    (metadata["body_site"].isin(["stool"])) &
    (metadata["disease"].isin([ "healthy"])) &
    (metadata["age_category"].isin(["adult"])) &
    (metadata["antibiotics_current_use"] != "yes")  # Escludere campioni con antibiotico "yes"

]
# Filtrare i metadati per i criteri specificati
filtered_metadata = metadata[
    (metadata["study_name"].isin(["DeFilippisF_2019"])) & 
    (metadata["diet"].isin(["omnivore", "vegan", "vegetarian"]))
]

# Ottenere i nomi delle colonne corrispondenti
selected_columns = filtered_metadata.iloc[:, 0].tolist()  # Prima colonna contiene i nomi delle colonne

# Filtrare il dataset metagenomico per le colonne selezionate
df = metagenomic_data[selected_columns]


In [57]:
#Select just the individuals that contains more than 100 values 
# Filtrare le colonne che hanno almeno 100 valori non nulli/non zero
df_species = df.loc[:, df.astype(bool).sum(axis=0) >= 100] 
# Filtrare il metadata per mantenere solo le righe che corrispondono alle colonne di df_species
filtered_metadata_for_species = filtered_metadata[filtered_metadata.iloc[:, 0].isin(df_species.columns)]

df_transposed = df_species.transpose()
df_transposed.replace(0, np.nan, inplace=True)
data = df_transposed.to_numpy()

# Save the filtered data to CSV files
#df_species.to_csv("C:/Users/spigno/Documents/GitHub/PopulationData/data_species.csv", index=False)
#filtered_metadata_for_species.to_csv("C:/Users/spigno/Documents/GitHub/PopulationData/filtermetadata_species.csv", index=False)

In [ ]:
 # Loop into all the columns/rows
for i in range(data.shape[1]):
    hist, bins = np.histogram(data[:, i], bins=np.logspace(-8, 1, 5), density=True)  # Bins logaritmici
    bins_mid = (bins[1:] + bins[:-1]) / 2
   
    plt.plot(bins_mid,hist)
    plt.xscale('log')
    plt.yscale('log')
    
plt.show()
      

In [ ]:
 # Loop into all the columns/rows
for i in range(data.shape[1]):

    plt.hist(data[:, i], bins=10)
    plt.show


In [ ]:
 # Loop into all the columns/rows
for i in range(data.shape[0]):

    plt.hist(data[i, :], bins=np.logspace(-8,1,10))
    plt.xscale('log')
    plt.yscale('log')
    plt.show

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np

# Create a PdfPages object to save all the plots
with PdfPages('histograms_diet_types_midpoints.pdf') as pdf:

    # Define the number of rows and columns for the grid (6x6)
    rows, cols = 6, 6
    
    # Create a figure and axes with a 6x6 grid
    fig, axes = plt.subplots(rows, cols, figsize=(18, 18))

    # Flatten the axes array to make it easier to index
    axes = axes.flatten()

    # Loop through each row and plot the bin midpoints in the grid
    for i in range(df_transposed.shape[0]):
        
        diet_type = filtered_metadata_for_species.iloc[i, 32]  # Adjust index if needed to get diet info
        # Extract the diet type for each column
        
        
        # Create histogram with logarithmic bins
        
        masked_data = data[i, :][~np.isnan(data[i, :])]

        hist, bins = np.histogram(masked_data, bins=np.logspace(-8,1,50), density=True)
        # Calculate the midpoints of the bins
        bins_mid = (bins[1:] + bins[:-1]) / 2

        # Plot the bin midpoints
        axes[i].scatter(bins_mid, hist, facecolors='none', edgecolors="r") 
        # Set the x and y axes to logarithmic scale
        axes[i].set_xscale('log')
        axes[i].set_yscale('log')
        # Set the title for each subplot with the diet type
        #axes[i].set_title(f'Diet: {diet_type}')
        axes[i].set_xlim(10e-7,10e1)
        axes[i].set_xlim(10e-2,10e2)
    # Adjust layout so titles and axes labels fit properly
    plt.tight_layout()

    # Save the entire grid of histograms into the PDF
    pdf.savefig(fig)

    # Close the figure to avoid memory issues
    plt.close(fig)


In [107]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np

# Create a PdfPages object to save all the plots
with PdfPages('SAD_bydiet.pdf') as pdf:
    # Definire i tipi di dieta
    diet_types = filtered_metadata_for_species["diet"].unique()

    # Define the number of rows and columns for the grid (1 row, 3 columns)
    rows, cols = 1, len(diet_types)

    # Create a figure and axes for the grid
    fig, axes = plt.subplots(rows, cols, figsize=(18, 6), squeeze=False)

    # Flatten the axes array to make it easier to index
    axes = axes.flatten()

    # Loop through each diet type and create a subplot
    for idx, diet in enumerate(diet_types):
        # Filter the data for the current diet type
        diet_indices = filtered_metadata_for_species["diet"] == diet
        diet_data = data[diet_indices, :]  # Select rows matching the current diet

        # Get the current axis
        ax = axes[idx]

        # Loop through each sample and plot its histogram
        for i in range(diet_data.shape[0]):
            # Remove NaN values
            masked_data = diet_data[i, :][~np.isnan(diet_data[i, :])]

            # Create histogram with logarithmic bins
            hist, bins = np.histogram(masked_data, bins=np.logspace(-5, 1, 50), density=True)

            # Calculate the midpoints of the bins
            bins_mid = (bins[1:] + bins[:-1]) / 2

            # Plot the bin midpoints as hollow markers
            ax.scatter(bins_mid, hist, facecolors='none', edgecolors='r', alpha=0.7)

        # Set the x and y axes to logarithmic scale
        ax.set_xscale('log')
        ax.set_yscale('log')
        #ax.set_xlim(min(bins)*0.9, 1)
        #ax.set_ylim(min(hist)*0.9, max(hist)*0.9)
        ax.set_xlabel('log(Abundance)')
        ax.set_ylabel('Density')
        ax.set_title(f'Diet: {diet}')


    # Adjust layout so titles and axes labels fit properly
    plt.tight_layout()

    # Save the entire grid of plots into the PDF
    pdf.savefig(fig)

    # Close the figure to avoid memory issues
    plt.close(fig)


In [77]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np

# Create a PdfPages object to save all the plots
with PdfPages('RankabundancePlot.pdf') as pdf:

    # Define the number of rows and columns for the grid (6x6)
    rows, cols = 6, 6
    
    # Create a figure and axes with a 6x6 grid
    fig, axes = plt.subplots(rows, cols, figsize=(18, 18))

    # Flatten the axes array to make it easier to index
    axes = axes.flatten()

    # Loop through each row and plot the bin midpoints in the grid
    for i in range(df_transposed.shape[0]):
        
        diet_type = filtered_metadata_for_species.iloc[i, 32]  # Adjust index if needed to get diet info
        # Extract the diet type for each column
        
        
        # Create histogram with logarithmic bins
        
        masked_data = data[i, :][~np.isnan(data[i, :])]
        vec = np.sort(masked_data)[::-1]
        ranks = np.arange(1, len(vec) + 1)
    
       # Plot the bin midpoints
        axes[i].scatter(ranks, vec) 
        # Set the x and y axes to logarithmic scale
        axes[i].set_xscale('log')
        axes[i].set_yscale('log')
        # Set the title for each subplot with the diet type
        axes[i].set_title(f'Diet: {diet_type}')
        axes[i].set_xlim(min(ranks),max(ranks)+1)
        axes[i].set_ylim(min(vec),max(vec)+1)
    # Adjust layout so titles and axes labels fit properly
    plt.tight_layout()

    # Save the entire grid of histograms into the PDF
    pdf.savefig(fig)

    # Close the figure to avoid memory issues
    plt.close(fig)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def rank_plot(x):
    # Ordinare il vettore in ordine decrescente
    vec = np.sort(x)[::-1]
    ranks = np.arange(1, len(vec) + 1)
    
    # Creare il plot log-log
    plt.figure()
    plt.scatter(ranks, vec)  # Ranks come x, valori ordinati come y
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('log(rank)')
    plt.ylabel('log(abundance)')
    
    # Mostrare il grafico
    plt.show()
    print(vec)


vec = data[0, :]  # Estrai la prima riga del dataset
rank_plot(vec)

In [84]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np

# Create a PdfPages object to save all the plots
with PdfPages('RankabundancePlot_bydiet.pdf') as pdf:
    # Definire i tipi di dieta
    diet_types = filtered_metadata_for_species["diet"].unique()

    # Define the number of rows and columns for the grid (1 row, 3 columns)
    rows, cols = 1, len(diet_types)

    # Create a figure and axes for the grid
    fig, axes = plt.subplots(rows, cols, figsize=(18, 6), squeeze=False)

    # Flatten the axes array to make it easier to index
    axes = axes.flatten()

    # Loop through each diet type and create a subplot
    for idx, diet in enumerate(diet_types):
        # Filter the data for the current diet type
        diet_indices = filtered_metadata_for_species["diet"] == diet
        diet_data = data[diet_indices, :]  # Select rows matching the current diet

        # Get the current axis
        ax = axes[idx]

        # Loop through each sample and plot its rank-abundance curve
        for i in range(diet_data.shape[0]):
            masked_data = diet_data[i, :][~np.isnan(diet_data[i, :])]  # Remove NaN values
            vec = np.sort(masked_data)[::-1]  # Sort in descending order
            ranks = np.arange(1, len(vec) + 1)  # Calculate ranks

            # Scatter plot for the current sample
            ax.scatter(ranks, vec, edgecolors="blue", facecolors='none')

        # Set the x and y axes to logarithmic scale
        ax.set_xscale('log')
        ax.set_yscale('log')
        ax.set_xlabel('log(rank)')
        ax.set_ylabel('log(abundance)')
        ax.set_title(f'Diet: {diet}')

    # Adjust layout so titles and axes labels fit properly
    plt.tight_layout()

    # Save the entire grid of plots into the PDF
    pdf.savefig(fig)

    # Close the figure to avoid memory issues
    plt.close(fig)

